Install the right PyTorch version for Joey NMT. Might have to restart the colab after installing Joey NMT.

In [1]:
!pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 763.5 MB 15 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.8.0+cu101 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.8.0+cu101 which is incompatible.


In [2]:
!pip install joeynmt

     |████████████████████████████████| 84 kB 3.7 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 7.1 MB 26.2 MB/s 
     |████████████████████████████████| 636 kB 68.2 MB/s 
     |████████████████████████████████| 375 kB 66.7 MB/s 
     |████████████████████████████████| 15.3 MB 104 kB/s 
     |████████████████████████████████| 105 kB 63.7 MB/s 
     |████████████████████████████████| 231 kB 72.8 MB/s 
     |████████████████████████████████| 55 kB 4.2 MB/s 
     |████████████████████████████████| 743 kB 48.7 MB/s 
  Created wheel for wrapt: filename=wrapt-1.11.1-cp37-cp37m-linux_x86_64.whl size=68432 sha256=96533a97c8c3a2dd58a1a2eff3cb5b712bc64590f9024f808a3f8b454104bc88
  Stored in directory: /root/.cache/pip/wheels/4e/58/9d/da8bad4545585ca52311498ff677647c95c7b690b3040171f8
Successfully built wrapt
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled si

## Download

In [3]:
# Đọc dữ liệu từ google drive và set đường dẫn 
from google.colab import drive
drive.mount('/content/gdrive')

path ='/content/gdrive/My Drive/Machine_Translation/'

vi_train = path + "indomain-news/train.vi"
vi_dev = path + "indomain-news/dev.vi"
vi_test = path + "indomain-news/tst.vi"

en_train = path + "indomain-news/train.en"
en_dev = path + "indomain-news/dev.en"
en_test = path + "indomain-news/tst.en"

Mounted at /content/gdrive


In [4]:
# Tiền xử lý
import string,re
def preprocess_nmt(text):
    """Preprocess the English-Vietnamese dataset."""
    def no_space(char, prev_char):
        return char in string.punctuation and prev_char != ' '

    # Replace non-breaking space with space, and convert uppercase letters to
    # lowercase ones
    text = text.replace('\u202f', ' ').replace('\xa0', ' ').lower()
    # Insert space between words and punctuation marks
    text = re.sub("\\s+", " ", text)
    out = [' ' + char if i > 0 and no_space(char, text[i - 1]) else char
           for i, char in enumerate(text)]
    return ''.join(out).strip()
import pandas as pd
def read_data_pandas(en_path, vi_path):
    source = []
    target = []
    skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
    with open(en_path, "r", encoding="utf8") as file:
        content = file.read().split("\n")
        for line in content:
          if line.strip() != "":    
              source.append(line.strip())      

    with open(vi_path, "r", encoding="utf8") as file:
        content = file.read().split("\n")
        for line in content:
          if line.strip() != "":
              target.append(line.strip())     
        
    df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])

    df["source_sentence"] = df["source_sentence"].apply(lambda x : preprocess_nmt(x))
    df["target_sentence"] = df["target_sentence"].apply(lambda x : preprocess_nmt(x))
    return df

df_train = read_data_pandas(en_train,vi_train)
df_dev = read_data_pandas(en_dev,vi_dev)
df_test = read_data_pandas(en_test,vi_test)

In [5]:
print(len(df_train),len(df_dev),len(df_test))

20000 1007 1220


In [6]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv
source_language = "en"
target_language = "vi"
with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in df_train.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in df_dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

with open("test."+source_language, "w") as src_file, open("test."+target_language, "w") as trg_file:
  for index, row in df_test.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")


In [7]:
!ls

dev.en	dev.vi	gdrive	sample_data  test.en  test.vi  train.en  train.vi


We'll only use a subset of dev and test data.

In [8]:
import os
isFile = os.path.isdir("data/en-vi/")
if isFile == False:
    os.makedirs("data/en-vi/")

!mv 'train.en' 'data/en-vi/train.en'
!mv 'dev.en' 'data/en-vi/dev.en'
!mv 'test.en' 'data/en-vi/test.en'

!mv 'train.vi' 'data/en-vi/train.vi'
!mv 'dev.vi' 'data/en-vi/dev.vi'
!mv 'test.vi' 'data/en-vi/test.vi'


The data is sentence-aligned, that means that source and target file contain one sentence per line which correspond to each other.

In [9]:
! head data/en-vi/dev.en


13 cases consisted of one individual each from thailand , china , morocco and india , two individuals each from saudi arabia , ethiopia , iran and three individuals from the uae were detected through early reporting by individuals .
therefore , koreans have fermented vegetables as food reserves for the winter .
last season , an incensed fan phoned nottinghamshire police reporting nani 's red card in the champions league match against real madrid as a 'crime ' .
later , as a result of the pandemic in italy , on march 8 , 2020 the italian government ordered all cinemas to be closed , for up to a month .
one purpose and guiding principle of the precede–proceed model is to direct initial attention to outcomes , rather than inputs .
since 1996 , the net worth of people under 35 has dropped by more than 34 percent .
the best example of echolocation is found in bats .
"in a statement to the journal nature biotechnology in february 2020 , us national institutes of health viral ecology unit chi

In [10]:
! head data/en-vi/dev.vi

13 trường hợp bao gồm một cá nhân mỗi người từ thái lan , trung quốc , morocco và ấn độ , hai cá nhân từ ả rập saudi , ethiopia , iran và ba cá nhân từ uae đã được phát hiện thông qua báo cáo sớm của các cá nhân .
do đó , người hàn quốc đã muối rau trái làm thức ăn dự trữ cho mùa đông .
"mùa trước , một cổ động viên giận dữ gọi điện đến cảnh sát nottinghamshire thông báo rằng chiếc thẻ đỏ của nani trong trận thuộc giải champions league với real madrid là một " "tội ác " " . "
sau đó , do hậu quả của đại dịch ở ý , vào ngày 8 tháng 3 năm 2020 , chính phủ ý đã ra lệnh đóng cửa tất cả các rạp chiếu phim , trong vòng một tháng .
một mục đích và nguyên tắc chỉ đạo của mô hình precede - proceed là để hướng sự chú ý ban đầu đến kết quả , chứ không phải là yếu tố đầu vào .
kể từ năm 1996 , khối tài sản ròng của những người dưới 35 tuổi đã giảm xuống hơn 34 % .
ví dụ tốt nhất của định vị bằng tiếng vang là ở loài dơi .
"trong một tuyên bố với tạp chí công nghệ sinh học tự nhiên vào tháng 2 nă

In [11]:
! wc -l data/en-vi/*

   1007 data/en-vi/dev.en
   1007 data/en-vi/dev.vi
   1220 data/en-vi/test.en
   1220 data/en-vi/test.vi
  20000 data/en-vi/train.en
  20000 data/en-vi/train.vi
  44454 total


## Subword model training

We will use the `subword_nmt` library to split words into subwords (BPE) according to their frequency in the training corpus.

In [12]:
import os

In [13]:
src_lang = 'en'
trg_lang = 'vi'
bpe_size = 4000
datadir = '/content/data/en-vi/'
name = f'{src_lang}_{trg_lang}_bpe{bpe_size}'


train_src_file = os.path.join(datadir, f'train.{src_lang}')
train_trg_file = os.path.join(datadir, f'train.{trg_lang}')
train_joint_file = os.path.join(datadir, f'train.{src_lang}-{trg_lang}')
dev_src_file = os.path.join(datadir, f'dev.{src_lang}')
dev_trg_file = os.path.join(datadir, f'dev.{trg_lang}')
test_src_file = os.path.join(datadir, f'test.{src_lang}')
test_trg_file = os.path.join(datadir, f'test.{trg_lang}')
src_files = {'train': train_src_file, 'dev': dev_src_file, 'test': test_src_file}
trg_files = {'train': train_trg_file, 'dev': dev_trg_file, 'test': test_trg_file}


vocab_src_file = os.path.join(datadir, f'vocab.{bpe_size}.{src_lang}')
vocab_trg_file = os.path.join(datadir, f'vocab.{bpe_size}.{trg_lang}')
bpe_file = os.path.join(datadir, f'bpe.codes.{bpe_size}')

Train a BPE model with 4000 symbols for both languages jointly.

In [14]:
! cat $train_src_file $train_trg_file > $train_joint_file

! subword-nmt learn-bpe \
  --input $train_joint_file \
  -s $bpe_size \
  -o $bpe_file

This file contains the merges of character sequences that subwords are made of.

In [15]:
! head $bpe_file

#version: 0.2
n g</w>
t h
t r
a n
c h
th e</w>
t i
i n
n h


We apply the learned BPE merges to training, development and test data.


In [16]:
src_bpe_files = {}
trg_bpe_files = {}
for split in ['train', 'dev', 'test']:
  src_input_file = src_files[split]
  trg_input_file = trg_files[split]
  src_output_file = src_input_file.replace(split, f'{split}.{bpe_size}.bpe')
  trg_output_file = trg_input_file.replace(split, f'{split}.{bpe_size}.bpe')
  src_bpe_files[split] = src_output_file
  trg_bpe_files[split] = trg_output_file

  ! subword-nmt apply-bpe \
    -c $bpe_file \
    < $src_input_file > $src_output_file

  ! subword-nmt apply-bpe \
    -c $bpe_file \
    < $trg_input_file > $trg_output_file


The subword-split data contains `@@ ` to indicate where words were split into subwords.

In [17]:
! head data/en-vi/dev.4000.bpe.en

13 cases con@@ s@@ is@@ ted of one individu@@ al each from thailand , china , mor@@ oc@@ co and india , two individuals each from sau@@ di ara@@ bia , e@@ thi@@ op@@ ia , iran and three individuals from the u@@ a@@ e were det@@ ected through early repor@@ ting by individuals .
the@@ re@@ fore , kore@@ ans have fer@@ men@@ ted ve@@ get@@ ab@@ les as food re@@ ser@@ ves for the win@@ ter .
last seas@@ on , an inc@@ en@@ sed f@@ an ph@@ oned no@@ t@@ ti@@ ng@@ ha@@ m@@ shi@@ re police repor@@ ting n@@ an@@ i 's red car@@ d in the ch@@ amp@@ i@@ ons lea@@ gue mat@@ ch against real mad@@ ri@@ d as a '@@ cri@@ me ' .
later , as a result of the pandemic in italy , on march 8 , 2020 the it@@ alian government or@@ dered all c@@ ine@@ mas to be closed , for up to a month .
one pur@@ po@@ se and gu@@ i@@ ding pr@@ inci@@ ple of the pre@@ ce@@ de@@ –@@ proce@@ ed mo@@ del is to direc@@ t initi@@ al atten@@ tion to out@@ comes , ra@@ ther than in@@ pu@@ ts .
since 199@@ 6 , the net worth of people 

In [18]:
! head data/en-vi/dev.4000.bpe.vi

13 trường hợp bao gồm một cá nhân mỗi người từ thái lan , trung quốc , mor@@ oc@@ co và ấn độ , hai cá nhân từ ả r@@ ập sau@@ di , e@@ thi@@ op@@ ia , iran và ba cá nhân từ u@@ a@@ e đã được phát hiện thông qua báo cáo sớm của các cá nhân .
do đó , người hàn quốc đã mu@@ ối ra@@ u trái làm thức ăn dự tr@@ ữ cho mùa đông .
"@@ mu@@ ̀@@ a trước , một cổ động viên gi@@ ận dữ gọi điện đến cảnh sát no@@ t@@ ti@@ ng@@ ha@@ m@@ shi@@ re thông báo rằng chiếc th@@ ẻ đỏ của n@@ an@@ i trong trận thuộc giải ch@@ amp@@ i@@ ons lea@@ gue với real mad@@ ri@@ d là một " "@@ tội ác " " . "
sau đó , do hậu quả của đại dịch ở ý , vào ngày 8 tháng 3 năm 2020 , chính phủ ý đã ra lệnh đóng cửa tất cả các r@@ ạp chiếu phim , trong vòng một tháng .
một mục đích và nguyên t@@ ắc chỉ đạo của mô hình pre@@ ce@@ de - proce@@ ed là để hướng sự chú ý ban đầu đến kết quả , ch@@ ứ không phải là yếu tố đầu vào .
kể từ năm 199@@ 6 , khối tài sản r@@ òng của những người dưới 35 tuổi đã giảm xuống hơn 34 % .
ví dụ tốt 

## Prepare the vocabulary

From the pre-processed training data, we extract the final vocabulary for the translation model. It should contain all subwords needed for representing the source and target training data.

In [19]:
! wget https://raw.githubusercontent.com/joeynmt/joeynmt/master/scripts/build_vocab.py

--2021-07-23 01:30:04--  https://raw.githubusercontent.com/joeynmt/joeynmt/master/scripts/build_vocab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2034 (2.0K) [text/plain]
Saving to: ‘build_vocab.py’

build_vocab.py      100%[===================>]   1.99K  --.-KB/s    in 0s      

2021-07-23 01:30:05 (34.8 MB/s) - ‘build_vocab.py’ saved [2034/2034]



In [20]:
vocab_src_file = src_bpe_files['train']
vocab_trg_file = trg_bpe_files['train']
bpe_vocab_file = os.path.join(datadir, f'joint.{bpe_size}bpe.vocab')

! python build_vocab.py  \
  $vocab_src_file $vocab_trg_file \
  --output_path $bpe_vocab_file

# Model configuration

Joey NMT reads model and training hyperparameters from a configuration file. We're generating this now to configure paths in the appropriate places. 

The configuration below builds a small Transformer model with shared embeddings between source and target language on the base of the subword vocabularies created above.

In [21]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{datadir}/train.{bpe_size}.bpe"
    dev:   "{datadir}/dev.{bpe_size}.bpe"
    test:  "{datadir}/test.{bpe_size}.bpe"
    level: "bpe"
    lowercase: False                
    max_sent_length: 300             # Extend to longer sentences.
    src_vocab: "{vocab_src_file}"
    trg_vocab: "{vocab_trg_file}"

testing:
    beam_size: 5
    alpha: 1.0
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "intl"            # `tokenize` option in sacrebleu.corpus_bleu() function (options include: "none" (use for already tokenized test data), "13a" (default minimal tokenizer), "intl" which mostly does punctuation and unicode, etc) 

training:
    #load_model: "models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # Alternative: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 32                     # Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 2000          # Set to at least once per epoch.
    logging_freq: 200
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: False               # Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 300
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True       # Requires joint vocabulary.
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 2
        num_heads: 2            # Increase to 8 for larger data.
        embeddings:
            embedding_dim: 512   # Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 512         # Increase to 512 for larger data.
        ff_size: 1024            # Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 2
        num_heads: 2              # Increase to 8 for larger data.
        embeddings:
            embedding_dim: 512    # Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 512         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, source_language=src_lang, target_language=trg_lang,
           datadir=datadir, vocab_src_file=bpe_vocab_file, 
           vocab_trg_file=bpe_vocab_file, bpe_size=bpe_size)
with open("transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Training

This will take a while. The log reports the training process, look out for the prints of example translations and the BLEU evaluation scores to get an impression of the current quality. 

The log is also stored in the model directory within this runtime (inspect files in the menu on the left). There you can also find a summary report of all validations. We'll also use TensorBoard to visualize the training progress on the go. This requires enabling Cookies in the browser.

After 12h at the latest, Colab will disconnect, so to make sure you're progress is not lost, download the checkpoints from the model directory from time to time. You'll later be able to reload them if model hyperparameters match.

In [22]:
!python -m joeynmt train transformer_en_vi_bpe4000.yaml

2021-07-23 01:30:39,281 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-23 01:30:39,343 - INFO - joeynmt.data - Loading training data...
2021-07-23 01:30:39,773 - INFO - joeynmt.data - Building vocabulary...
2021-07-23 01:30:40,085 - INFO - joeynmt.data - Loading dev data...
2021-07-23 01:30:40,100 - INFO - joeynmt.data - Loading test data...
2021-07-23 01:30:40,123 - INFO - joeynmt.data - Data loaded.
2021-07-23 01:30:40,124 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-23 01:30:40,367 - INFO - joeynmt.model - Enc-dec model built.
2021-07-23 01:30:40.607120: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-23 01:30:42,410 - INFO - joeynmt.training - Total params: 12763648
2021-07-23 01:30:46,024 - INFO - joeynmt.helpers - cfg.name                           : en_vi_bpe4000_transformer
2021-07-23 01:30:46,024 - INFO - joeynmt.helpers - cfg.data.src                       : 

In [23]:
!python -m joeynmt test transformer_en_vi_bpe4000.yaml

2021-07-23 02:02:28,347 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-23 02:02:28,348 - INFO - joeynmt.data - Building vocabulary...
2021-07-23 02:02:28,663 - INFO - joeynmt.data - Loading dev data...
2021-07-23 02:02:28,679 - INFO - joeynmt.data - Loading test data...
2021-07-23 02:02:28,699 - INFO - joeynmt.data - Data loaded.
2021-07-23 02:02:28,736 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 18000 (with beam_size)
2021-07-23 02:02:32,364 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-23 02:02:32,594 - INFO - joeynmt.model - Enc-dec model built.
2021-07-23 02:02:32,650 - INFO - joeynmt.prediction - Decoding on dev set (/content/data/en-vi//dev.4000.bpe.vi)...
2021-07-23 02:03:09,987 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2021-07-23 02:03:09,988 - WARNING - sacrebleu - It looks like you forgot to detokenize your test data, which may hurt your score.
2021-07-23 02:03

In [24]:
!python -m joeynmt test transformer_en_vi_bpe4000.yaml --output_path predictions

2021-07-23 02:04:10,835 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-23 02:04:10,835 - INFO - joeynmt.data - Building vocabulary...
2021-07-23 02:04:11,134 - INFO - joeynmt.data - Loading dev data...
2021-07-23 02:04:11,150 - INFO - joeynmt.data - Loading test data...
2021-07-23 02:04:11,169 - INFO - joeynmt.data - Data loaded.
2021-07-23 02:04:11,199 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 18000 (with beam_size)
2021-07-23 02:04:14,760 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-23 02:04:14,998 - INFO - joeynmt.model - Enc-dec model built.
2021-07-23 02:04:15,057 - INFO - joeynmt.prediction - Decoding on dev set (/content/data/en-vi//dev.4000.bpe.vi)...
2021-07-23 02:04:55,631 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2021-07-23 02:04:55,631 - WARNING - sacrebleu - It looks like you forgot to detokenize your test data, which may hurt your score.
2021-07-23 02:04